In [16]:
print("Hello World")

Hello World


In [17]:
pip install -r requirements.txt


Note: you may need to restart the kernel to use updated packages.


# Visualize Dataset

In [18]:
import pandas as pd 

df_ = pd.read_csv("../data/Air Quality Ho Chi Minh City.csv")
df = df_.copy()
df.head()

,date,Station_No,TSP,PM2.5,O3,CO,NO2,SO2,Temperature,Humidity
0,23-02-2021 21:00,1,32.935714,15.604762,55.431381,1330.451429,112.740762,393.000000,28.361905,63.188095
1,23-02-2021 22:00,1,30.932353,14.594118,58.197176,1200.603529,112.366471,377.588235,28.320588,63.773529
2,23-02-2021 23:00,1,27.645000,13.436667,55.029433,1177.897000,112.700433,372.476667,28.336667,64.205000
3,24-02-2021 00:00,1,24.380000,12.365000,54.767700,1267.476000,112.480867,389.070000,28.305000,64.735000
4,24-02-2021 01:00,1,22.521667,11.636667,53.786200,1322.293000,114.331500,393.000000,28.300000,65.188333


In [19]:
df['Station_No'].unique()

array([1, 2, 3, 4, 5, 6], dtype=int64)

Tách date thành => 3 cột Ngày Tháng Năm nếu cần thiết trong mô hình

In [20]:
df['date']= pd.to_datetime(df['date'], dayfirst=True)
print(df['date'].head())

0   2021-02-23 21:00:00
1   2021-02-23 22:00:00
2   2021-02-23 23:00:00
3   2021-02-24 00:00:00
4   2021-02-24 01:00:00
Name: date, dtype: datetime64[ns]


In [21]:
df.isna().sum()

date               0
Station_No         0
TSP               60
PM2.5              0
O3             10610
CO              9065
NO2             5666
SO2            11006
Temperature     4437
Humidity        4432
dtype: int64

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52548 entries, 0 to 52547
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         52548 non-null  datetime64[ns]
 1   Station_No   52548 non-null  int64         
 2   TSP          52488 non-null  float64       
 3   PM2.5        52548 non-null  float64       
 4   O3           41938 non-null  float64       
 5   CO           43483 non-null  float64       
 6   NO2          46882 non-null  float64       
 7   SO2          41542 non-null  float64       
 8   Temperature  48111 non-null  float64       
 9   Humidity     48116 non-null  float64       
dtypes: datetime64[ns](1), float64(8), int64(1)
memory usage: 4.0 MB


In [23]:
df.shape

(52548, 10)

In [24]:
df.duplicated().sum()

0

In [25]:
df.fillna(0,inplace=True) #fill gia tri null = 0

In [26]:
df.isnull().sum()

date           0
Station_No     0
TSP            0
PM2.5          0
O3             0
CO             0
NO2            0
SO2            0
Temperature    0
Humidity       0
dtype: int64

# Chia Tập Dữ Liệu #

In [27]:
df['date']

0       2021-02-23 21:00:00
1       2021-02-23 22:00:00
2       2021-02-23 23:00:00
3       2021-02-24 00:00:00
4       2021-02-24 01:00:00
                ...        
52543   2022-06-21 13:00:00
52544   2022-06-21 14:00:00
52545   2022-06-21 15:00:00
52546   2022-06-21 16:00:00
52547   2022-06-21 17:00:00
Name: date, Length: 52548, dtype: datetime64[ns]

từ 23/02/2021 đến 21/06/2022 <br>
Chia date thành year, month, day, hour

In [28]:
# df['year'] = df['date'].dt.year
# df['month'] = df['date'].dt.month
# df['day'] = df['date'].dt.day
# df['hour'] = df['date'].dt.hour
# df = df.drop(columns=['date'])


Chia 6 khu vực thành 6 tệp json

In [29]:

for station_id in df['Station_No'].unique():
    df_station = df[df['Station_No'] == station_id]
    
    df_station.to_csv(f'../data/station_{station_id}.csv', index=False, encoding='utf-8-sig')

# Model #

In [56]:
df1_ = pd.read_csv('../data/station_1.csv')
df1 = df1_.copy()
df1 = df1.sort_values('date')

In [33]:
df1.head()

,date,Station_No,TSP,PM2.5,O3,CO,NO2,SO2,Temperature,Humidity
0,2021-02-23 21:00:00,1,32.935714,15.604762,55.431381,1330.451429,112.740762,393.000000,28.361905,63.188095
1,2021-02-23 22:00:00,1,30.932353,14.594118,58.197176,1200.603529,112.366471,377.588235,28.320588,63.773529
2,2021-02-23 23:00:00,1,27.645000,13.436667,55.029433,1177.897000,112.700433,372.476667,28.336667,64.205000
3,2021-02-24 00:00:00,1,24.380000,12.365000,54.767700,1267.476000,112.480867,389.070000,28.305000,64.735000
4,2021-02-24 01:00:00,1,22.521667,11.636667,53.786200,1322.293000,114.331500,393.000000,28.300000,65.188333


<h1>Chuẩn hóa dữ liệu</h1>

In [35]:
from sklearn.preprocessing import MinMaxScaler

features = ['TSP','O3','CO','NO2', 'SO2', 'Temperature', 'Humidity']
target = ['PM2.5']

scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()

X_scaler = scaler_X.fit_transform(df1[features])
Y_scaler = scaler_Y.fit_transform(df1[target])

Chuỗi dữ liệu cho LSTM

In [39]:
import numpy as np
def create_sequences(X, y, seq_len):
    X_seq, y_seq = [], []
    for i in range(seq_len, len(X)):
        X_seq.append(X[i-seq_len:i])
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)

X_seq , y_seq = create_sequences(X_scaler, Y_scaler, 100)

Chia data

In [41]:
train_size = int(0.7 * len(X_seq))       # 70% huan luyen
val_size = int(0.15 * len(X_seq))        # 15% kiem tra model
test_size = len(X_seq) - train_size - val_size  

X_train = X_seq[:train_size]
y_train = y_seq[:train_size]

X_val = X_seq[train_size:train_size + val_size]
y_val = y_seq[train_size:train_size + val_size]

X_test = X_seq[train_size + val_size:]
y_test = y_seq[train_size + val_size:]

Chuyển sang tensor

In [42]:
import torch
from torch.utils.data import TensorDataset, DataLoader

def create_loader(X, y, batch_size =32, shuffle= False):
    X_tensor = torch.tensor(X,  dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32).view(-1,1)
    dataset = TensorDataset(X_tensor, y_tensor)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

train_loader = create_loader(X_train, y_train, batch_size=32, shuffle=True)
val_loader   = create_loader(X_val, y_val, batch_size=32, shuffle=False)
test_loader  = create_loader(X_test, y_test, batch_size=32, shuffle=False)

Mô hình LSTM

x: batch data đầu vào <br>

batch = số lượng mẫu trong một batch 
<br>
seq_length = số bước thời gian (24 giờ gần nhất)
<br>
input_dim = số đặc trưng mỗi thời điểm (PM2.5)



In [ ]:
import torch.nn as nn 

class LSTM_Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers = 1):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size,1) # dau ra 1 gtri PM2.5
        
    def forward(self, x):
        # x: (batch, seq_length,, input_dim)
        lstm_out, (h_n, c_n) = self.lstm(x)
        # output tại thời điểm cuối
        last_out = lstm_out[:, -1, :]
        output = self.fc(last_out)
        return output


Huấn Luyện Mô Hình

In [47]:
input_dim = len(features)
hidden_dim = 64
model = LSTM_Model(input_dim, hidden_dim)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

epochs = 10

for epoch in range(epochs):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = loss_fn(y_pred,y_batch)
        loss.backward() #lan truyền ngược (tự động tình đạo hàm theo tham số mô hình)
        optimizer.step() #cập nhật trọng số
        train_loss += loss.item()
        
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_val_batch, y_val_batch in val_loader:
            y_val_pred = model(X_val_batch)
            val_loss += loss_fn(y_val_pred, y_val_batch).item()
    
    print(f"Epoch {epoch+1:2} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
        

Epoch  1 | Train Loss: 0.3572 | Val Loss: 0.0364
Epoch  2 | Train Loss: 0.1690 | Val Loss: 0.0259
Epoch  3 | Train Loss: 0.1492 | Val Loss: 0.0189
Epoch  4 | Train Loss: 0.1377 | Val Loss: 0.0162
Epoch  5 | Train Loss: 0.1312 | Val Loss: 0.0205
Epoch  6 | Train Loss: 0.1291 | Val Loss: 0.0146
Epoch  7 | Train Loss: 0.1210 | Val Loss: 0.0137
Epoch  8 | Train Loss: 0.1195 | Val Loss: 0.0132
Epoch  9 | Train Loss: 0.1162 | Val Loss: 0.0127
Epoch 10 | Train Loss: 0.1113 | Val Loss: 0.0110


Cải thiện: <br>
Tăng hidden_size <br>

Tăng num_layers
<br>
Thêm Dropout
<br>
Huấn luyện lâu hơn (20–30 epoch) + dùng early stopping



Đánh giá tập test

In [48]:
model.eval()
test_loss = 0
preds = []
targets = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)
        test_loss += loss_fn(y_pred, y_batch).item()
        preds.append(y_pred.numpy())
        targets.append(y_batch.numpy())

print(f"\n Test Loss: {test_loss:.4f}")


 Test Loss: 0.0171


$$
\text{Loss} = \frac{1}{N} \sum_{i=1}^{N} \left( \hat{y}_i - y_i \right)^2
$$


In [50]:
torch.save(model.state_dict(), "../model/lstm_model.pth") # lưu mô hình

In [51]:
model = LSTM_Model(input_dim, hidden_dim)
model.load_state_dict(torch.load("../model/lstm_model.pth"))

X_true = X_seq
y_true = y_seq
true_data = create_loader(X_true, y_true, batch_size=32, shuffle=True)

model.eval()
loss_fn = nn.MSELoss()

test_loss_true = 0
preds_true = []
targets_true = []

with torch.no_grad():
    for X_batch, y_batch in true_data:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        test_loss_true += loss.item()

        preds_true.append(y_pred.numpy())
        targets_true.append(y_batch.numpy())


preds_true = np.concatenate(preds_true)
targets_true = np.concatenate(targets_true)

print(f" Test Loss (MSE): {test_loss_true:.4f}")

 Test Loss (MSE): 0.1377


7792


date           7892
Station_No     7892
TSP            7892
PM2.5          7892
O3             7892
CO             7892
NO2            7892
SO2            7892
Temperature    7892
Humidity       7892
dtype: int64